In [1]:
from Artist.progressive_trainer import ProgressiveGAN

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
iterations = [100000] * 6
batch_sizes = [2 ** n for n in range(7, 1, -1)]

gan = ProgressiveGAN(
    art_type="celeba-hq", 
    name="AI_Artist_Type_1", 
    json_path="settings.json",
    batch_sizes=batch_sizes, 
    n_label=1, 
    display_interval=100, 
    n=1000,
    iterations=iterations, 
    display_length=64, 
    initial_step=0, 
    gpu=True
)

In [3]:
checkpoint = ModelCheckpoint(save_top_k=None, monitor=None)

trainer = Trainer(
    gpus=1,
    auto_select_gpus=True,
    callbacks=[checkpoint]
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [4]:
trainer.fit(gan)


  | Name          | Type          | Params
------------------------------------------------
0 | loss          | Loss          | 0     
1 | generator     | Generator     | 22.9 M
2 | discriminator | Discriminator | 25.2 M
------------------------------------------------
48.1 M    Trainable params
0         Non-trainable params
48.1 M    Total params


Training: 0it [00:00, ?it/s]

TypeError: len() of a 0-d tensor